In [24]:
# 将规范的数组, 转化为树型结构
class TreeNode:
    def __init__(self, x):
        self.val = x
        self.left = None
        self.right = None

class Node:
    def __init__(self, val, children=[]):
        self.val = val
        self.children = children
        
class ListNode:
    def __init__(self, x):
        self.val = x
        self.next = None
        
def array2link(arr):
    if not arr:
        return None
    node = ListNode(arr[0])
    root = node
    for i in range(1, len(arr)):
        node.next = ListNode(arr[i])
        node = node.next
    return root
        
def array2tree(arr):
    n = arr.pop(0)
    root = None if n is None else TreeNode(n)
    queue = [root]
    while arr and queue:
        node = queue.pop(0)
        if node is None:
            continue
        
        n1 = arr.pop(0)
        node.left = None if n1 is None else TreeNode(n1)
        queue.append(node.left)
        if arr:
            n2 = arr.pop(0)
            node.right = None if n2 is None else TreeNode(n2)
            queue.append(node.right)
    return root

In [1]:
"""
71. 简化路径
https://leetcode-cn.com/problems/simplify-path/
解题思路
  1. 使用"/"进行分割字符串后, 过滤掉空的元素.
  2. 使用堆栈存储, 如果遇到".."则弹出栈顶; 否则, 如果不等于".", 则压入堆栈.
"""
def simplifyPath(path: str) -> str:
    # 使用"/"进行分割, 并且过滤掉空的字符
    path = list(filter(lambda a: a, path.split('/')))
    s = []
    for p in path:
        # 如果为"..", 并且堆栈不为空, 则弹出栈顶元素
        if p == '..' and s:
            s.pop()
        # 如果不为".", "..", 则为正常的路径, 则压入堆栈中
        elif p not in ['.', '..']:
            s.append(p)

    # 格式化输出路径
    return '/' + '/'.join(s)

# 简单测试
# /
print(simplifyPath("/../"))
# /home/foo
print(simplifyPath("/home//foo/"))
# /c
print(simplifyPath("/a/./b/../../c/"))
# /c
print(simplifyPath("/a/../../b/../c//.//"))
# /a/b/c
print(simplifyPath("/a//b////c/d//././/.."))

/
/home/foo
/c
/c
/a/b/c


In [3]:
"""
94. 二叉树的中序遍历
https://leetcode-cn.com/problems/binary-tree-inorder-traversal/
解题思路
    1: 递归方法
    2: 迭代方法
"""   
def inorderTraversal(root: TreeNode):
    r = []
    def tree(node):
        nonlocal r
        if not node:
            return
        tree(node.left)
        r.append(node.val)
        tree(node.right)
    
    tree(root)
    return r

def inorderTraversal1(root: TreeNode):
    r, s, cur = [], [], root
    while cur or s:
        # 将左子树压入堆栈中
        while cur:
            s.append(cur)
            cur = cur.left
        cur = s.pop()
        r.append(cur.val)
        # 将左子树对应的右子树进行遍历
        cur = cur.right
    
    return r

t1, t2, t3 = TreeNode(1), TreeNode(2), TreeNode(3)
t1.right = t2
t2.left = t3
print(inorderTraversal(t1))
print(inorderTraversal1(t1))
        

[1, 3, 2]
[1, 3, 2]


In [7]:
"""
103. 二叉树的锯齿形层次遍历
https://leetcode-cn.com/problems/binary-tree-zigzag-level-order-traversal/
解题思路
  1. 先层序遍历, 然后对偶数行的层进行倒序
"""     
def zigzagLevelOrder(root: TreeNode):
    r = []
    def tree(node, level=0):
        nonlocal r
        if not node:
            return
        if len(r) <= level:
            r.append([])
        tree(node.left, level + 1)
        r[level].append(node.val)
        tree(node.right, level + 1)
        
    tree(root)
    
    for i in range(len(r)):
        if i % 2 == 1:
            r[i].reverse()
    
    return r
        
# 简单测试
t = array2tree([3,9,20,None,None,15,7])
print(zigzagLevelOrder(t))

[[3], [20, 9], [15, 7]]


In [8]:
"""
144. 二叉树的前序遍历
https://leetcode-cn.com/problems/binary-tree-preorder-traversal/
解题思路
  1. 递归
  2. 迭代
"""     
def preorderTraversal(root: TreeNode):
    r = []
    def tree(node):
        nonlocal r
        if not node:
            return
        r.append(node.val)
        tree(node.left)
        tree(node.right)
        
    tree(root)
    
    return r

def preorderTraversal1(root: TreeNode):
    if not root:
        return []
    r, s = [], [root]
    while s:
        node = s.pop()
        r.append(node.val)
        if node.right:
            s.append(node.right)
        if node.left:
            s.append(node.left)
    
    return r

t = array2tree([1,None,2,3])
print(preorderTraversal(t))
print(preorderTraversal1(t))

[1, 2, 3]
[1, 2, 3]


In [9]:
"""
150. 逆波兰表达式求值
https://leetcode-cn.com/problems/evaluate-reverse-polish-notation/
解题思路
  1. 使用堆栈实现, 如果遇到"+-*/", 则弹出堆栈栈顶两个元素进行计算, 然后将计算结果在压入堆栈中.
"""
def evalRPN(tokens) -> int:
    s = []
    for t in tokens:
        if t in '+-*/':
            v1 = s.pop()
            v2 = s.pop()
            if t == '+':
                s.append(v1 + v2)
            elif t == '-':
                s.append(v2 - v1)
            elif t == '*':
                s.append(v2 * v1)
            else:
                # 如果遇到负数, 则不能直接除, 而是转为整数相除后, 再取负
                if (v1 < 0 and v2 > 0) or (v1 > 0 and v2 < 0):
                    s.append(-(abs(v2) // abs(v1)))
                else:
                    s.append(v2 // v1)
        else:
            s.append(int(t))
            
    return s[0]

# 简单测试
# 9
print(evalRPN(["2","1","+","3","*"]))
# 6
print(evalRPN(["4", "13", "5", "/", "+"]))
# 22
print(evalRPN(["10", "6", "9", "3", "+", "-11", "*", "/", "*", "17", "+", "5", "+"]))

9
6
22


In [10]:
"""
173. 二叉搜索树迭代器
https://leetcode-cn.com/problems/binary-search-tree-iterator/
解题思路
  1. 将二叉树存储为有序数组.
  2. 使用二叉树原生遍历的办法
"""
class BSTIterator:
    def __init__(self, root: TreeNode):
        self.r = []
        # 将二叉搜索树中序遍历, 则数组为有序
        def tree(node):
            if not node:
                return
            tree(node.left)
            self.r.append(node.val)
            tree(node.right)
        
        tree(root)
        
    
    def next(self) -> int:
        return self.r.pop(0)

    
    def hasNext(self) -> bool:
        return bool(self.r)
    
class BSTIterator1:
    def __init__(self, root: TreeNode):
        self.root = root
        
    
    def next(self) -> int:
        node = self.root
        # 如果没有左孩子, 则根节点为最小节点
        if not node.left:
            self.root = node.right
            return node.val
        
        pre = None
        # 找到最小节点, 然后将其右孩子复制到当前节点
        while node.left:
            pre = node
            node = node.left
        pre.left = node.right
        return node.val
    
    
    def hasNext(self) -> bool:
        return bool(self.root)
    

# 简单测试
t = array2tree([7,3,15,None,None,9,20])
iterator = BSTIterator(t)
# 3
print(iterator.next())
# 7
print(iterator.next())
# True
print(iterator.hasNext())
# 9
print(iterator.next())
# True
print(iterator.hasNext())
# 15
print(iterator.next())
# True
print(iterator.hasNext())
# 20
print(iterator.next())

iterator1 = BSTIterator(t)
# 3
print(iterator1.next())
# 7
print(iterator1.next())
# True
print(iterator1.hasNext())
# 9
print(iterator1.next())
# True
print(iterator1.hasNext())
# 15
print(iterator1.next())
# True
print(iterator1.hasNext())
# 20
print(iterator1.next())

3
7
True
9
True
15
True
20
3
7
True
9
True
15
True
20


In [1]:
"""
215. 数组中的第K个最大元素
https://leetcode-cn.com/problems/kth-largest-element-in-an-array/submissions/
解题思路: 使用heapq
"""
def findKthLargest(nums, k):
    import heapq
    s = heapq.nlargest(k, nums)
    return s[-1]

print(findKthLargest([3,2,1,5,6,4], 2))
print(findKthLargest([3,2,3,1,2,4,5,5,6], 4))

5
4


In [4]:
"""
264. 丑数II
https://leetcode-cn.com/problems/ugly-number-ii/submissions/
"""
def nthUglyNumber(n):
    ugly = [1]
    i2 = i3 = i5 = 0
    while len(ugly) < n:
        # 这里会有序的将i2, i3, i5增加到正确的索引
        while ugly[i2] * 2 <= ugly[-1]: i2 += 1
        while ugly[i3] * 3 <= ugly[-1]: i3 += 1
        while ugly[i5] * 5 <= ugly[-1]: i5 += 1
        # 取当前乘积的最小值, 写入数组中
        ugly.append(min(ugly[i2] * 2, ugly[i3] * 3, ugly[i5] * 5))
    return ugly[-1]

def nthUglyNumber1(n):
    ugly = sorted(2**a * 3**b * 5**c
                 for a in range(32) for b in range(20) for c in range(14))
    return ugly[n - 1]

print(nthUglyNumber(1234))
print(nthUglyNumber1(1234))

210937500
210937500


In [5]:
"""
313. 超级丑数
https://leetcode-cn.com/problems/super-ugly-number/submissions/
"""
def nthSuperUglyNumber(n, primes):
    ugly = [1]
    indexs = [0 for _ in range(len(primes))]
    while len(ugly) < n:
        for i in range(len(indexs)):
            while ugly[indexs[i]] * primes[i] <= ugly[-1]:
                indexs[i] += 1
        ugly.append(min([ugly[indexs[i]] * primes[i] for i in range(len(indexs))]))
    return ugly[-1]

print(nthSuperUglyNumber(1234, [2,7,13,19]))

282475249


In [11]:
"""
331. 验证二叉树的前序序列化
https://leetcode-cn.com/problems/verify-preorder-serialization-of-a-binary-tree/
解题思路
  1. 遇到两个"#"号, 则将"叶子节点 + # + #"设置为一个#号即可. 最后字符串会成空.
"""
def isValidSerialization(preorder: str) -> bool:
    preorder = preorder.split(',')
    # 为空的情况
    if not preorder:
        return True
    # 头结点为#的情况
    if preorder[0] == '#':
        return bool(len(preorder) == 1)
    
    i = 1
    while i < len(preorder) - 1:
        if preorder[i - 1] != '#' and preorder[i] == '#' and preorder[i + 1] == '#':
            preorder[i - 1:i + 2] = '#'
            # 这步至关重要, 只要合并为#, 则需要重头开始
            i = 0
        i += 1
    
    return len(preorder) == 1 and preorder[0] == '#'

# 简单测试
# True
print(isValidSerialization("9,3,4,#,#,1,#,#,2,#,6,#,#"))
# False
print(isValidSerialization("1,#"))
# False
print(isValidSerialization("9,#,#,1"))
# True
print(isValidSerialization("#"))
# False
print(isValidSerialization(""))
# False
print(isValidSerialization("1,#,#,#,#"))    

True
False
False
True
False
False


In [12]:
"""
341. 扁平化嵌套列表迭代器
https://leetcode-cn.com/problems/flatten-nested-list-iterator/
解题思路
  1. 定义help函数, 将元素为数组的, 扩展开来.
  2. 注释内的代码为: LeetCode上配合NestedInteger类所作出的修改.
"""
class NestedIterator:
    def __init__(self, nestedList):
        def help(arr):
            _arr = []
            for a in arr:
                # if a.isInteger():
                if type(a) != list:
                    _arr.append(a)
                else:
                    # _arr.extend(help(a.getList()))
                    _arr.extend(help(a))
            return _arr
        self.nestedList = help(nestedList)        
        
    
    def next(self):
        return self.nestedList.pop(0)
    
    
    def hasNext(self):
        return bool(self.nestedList)
    
# 简单测试
arr, v = NestedIterator([[1,1],2,[1,1]]), []
while arr.hasNext():
    v.append(arr.next())
print(v)

[1, 1, 2, 1, 1]


In [6]:
"""
347. 前K个高频元素
https://leetcode-cn.com/problems/top-k-frequent-elements/submissions/
"""
import collections
import heapq
def topKFrequent(nums, k):
    d = collections.defaultdict(int)
    s = []
    for n in nums:
        d[n] += 1
    for key, value in d.items():
        s.append((value, key))
    r = heapq.nlargest(k, s)
    return [_r[1] for _r in r]

print(topKFrequent([1,1,1,2,2,3], 2))
print(topKFrequent([1], 1))
print(topKFrequent([1,2,3,4,5,6,5,4,3,5,6,2,3,4,5], 3))

[1, 2]
[1]
[5, 4, 3]


In [ ]:
"""
355. 设计推特
https://leetcode-cn.com/problems/design-twitter/
"""
class Twitter:

    def __init__(self):
        self.timer = itertools.count(step=-1)
        self.tweets = collections.defaultdict(collections.deque)
        self.followees = collections.defaultdict(set)
        

    def postTweet(self, userId: int, tweetId: int) -> None:
        self.tweets[userId].appendleft((next(self.timer), tweetId))
        """
        Compose a new tweet.
        """
        

    def getNewsFeed(self, userId: int) -> List[int]:
        tweets = heapq.merge(*(self.tweets[u] for u in self.followees[userId] | {userId}))
        return [t for _, t in itertools.islice(tweets, 10)]

    def follow(self, followerId: int, followeeId: int) -> None:
        self.followees[followerId].add(followeeId)
        

    def unfollow(self, followerId: int, followeeId: int) -> None:
        self.followees[followerId].discard(followeeId)
     

In [8]:
"""
373. 查找和最小的K对数字
https://leetcode-cn.com/problems/find-k-pairs-with-smallest-sums/submissions/
"""
def kSmallestPairs(nums1, nums2, k):
    heap = []
    for n1 in nums1:
        for n2 in nums2:
            if len(heap) < k:
                # 堆的顶部元素存储的是最小的, 所以这里存储负数, 来代表最大的
                heapq.heappush(heap, (-n1-n2, [n1, n2]))
            else:
                # 说明存在更小的元素, 所以需要pop出最大的元素, 存储更小的元素
                if heap and -heap[0][0] > n1 + n2:
                    heapq.heappop(heap)
                    heapq.heappush(heap, (-n1-n2, [n1, n2]))
                else:
                    break
    return list(reversed([heapq.heappop(heap)[1] for _ in range(k) if heap]))

print(kSmallestPairs([1,1,2], [1,2,3], 10))

[[1, 1], [1, 1], [2, 1], [1, 2], [1, 2], [2, 2], [1, 3], [1, 3], [2, 3]]


In [9]:
"""
378. 有序矩阵中第K小的元素
https://leetcode-cn.com/problems/kth-smallest-element-in-a-sorted-matrix/submissions/
"""
# 373题目类似的
def kthSmallest(matrix, k):
    heap, m, n = [], len(matrix), len(matrix[0])
    for i in range(m):
        for j in range(n):
            if len(heap) < k:
                heapq.heappush(heap, -matrix[i][j])
            else:
                if heap and -heap[0] > matrix[i][j]:
                    heapq.heappop(heap)
                    heapq.heappush(heap, -matrix[i][j])
                else:
                    break
    return -heapq.heappop(heap)

matrix = [
    [1, 5, 9],
    [10, 11, 13],
    [12, 13, 15]
]
print(kthSmallest(matrix, 8))

13


In [ ]:
"""
385. 迷你语法分析器
https://leetcode-cn.com/problems/mini-parser/
解题思路
  1. 不能使用eval.(使用堆栈)
  2. 答案来自: https://leetcode.com/problems/mini-parser/discuss/335124/Python-stack
"""
def deserialize(s):
    stack = []
    elem = None
    hasNum, sgn, num = False, 1, 0
    for x in s:
        if x == '[':
            stack.append(NestedInteger())
        elif x == '-':
            sgn = -1
        elif x.isdigit():
            hasNum = True
            num = num * 10 + int(x)
        else:
            if hasNum:
                stack[-1].add(NestedInteger(sgn * num))
                hasNum, sgn, num = False, 1, 0
            elif elem is not None:
                stack[-1].add(elem)
                elem = None
            if x == ']':
                elem = stack.pop()
                
    if hasNum:
        return NestedInteger(sgn * num)
    return elem

In [14]:
"""
394. 字符串解码
https://leetcode-cn.com/problems/decode-string/
"""
def decodeString(s: str) -> str:
    count, letters, stack = '', '', []
    for i in range(len(s)):
        # 数字压入堆栈
        if s[i].isdigit():
            # 如果前面出现字母, 则将字母压入堆栈
            if letters:
                stack.append(letters)
                letters = ''
            count += s[i]
            continue
        elif count:
            # 如果前面出现过数字, 则将数字压入堆栈
            stack.append(count)
            count = ''
        # 字母压入堆栈
        if s[i].isalpha():
            letters += s[i]
            continue
        elif letters:
            stack.append(letters)
            letters = ''
            
        if '[' == s[i]:
            stack.append('[')
        elif ']' == s[i]:
            # 字母
            _letters = []
            while stack[-1] != '[':
                _letters.append(stack.pop())
            _letters = ''.join(reversed(_letters))
            # 弹出'['
            stack.pop()
            # 数字
            _count = int(stack.pop())
            if stack and stack[-1] != '[':
                stack[-1] += _letters * _count
            else:
                stack.append(_letters * _count)
                
    return ''.join(stack) + letters
      
# 简单测试
# aaabcbc
print(decodeString("3[a]2[bc]"))
# accaccacc
print(decodeString("3[a2[c]]"))
# abcabccdcdcdef
print(decodeString("2[abc]3[cd]ef"))

aaabcbc
accaccacc
abcabccdcdcdef


In [11]:
"""
402. 移掉K位数字
https://leetcode-cn.com/problems/remove-k-digits/
解题思路
  1. 遍历字符串, 只要s[i] > s[i + 1], 则需要移除s[i]
"""
def removeKdigits(num: str, k: int) -> str:
    if k >= len(num):
        return "0"
    while k > 0:
        i, length = 0, len(num)
        while i + 1 < length and num[i] <= num[i + 1]:
            i += 1
        num = num[:i] + num[i + 1:]
        k -= 1
   
    num = num.lstrip("0") or "0"
    return num

# 简单测试
# 1219
print(removeKdigits("1432219", 3))
# 200
print(removeKdigits("10200", 1))
# 0
print(removeKdigits("10", 2))
# 0
print(removeKdigits("10", 1))

1219
200
0
0


In [12]:
"""
451. 根据字符出现频率排序
https://leetcode-cn.com/problems/sort-characters-by-frequency/submissions/
"""
def frequencySort(s):
    d, h = collections.defaultdict(int), []
    for _s in s:
        d[_s] += 1
    for k, v in d.items():
        heapq.heappush(h, (-v, k * v))
    return "".join([heapq.heappop(h)[1] for _ in range(len(h))])

print(frequencySort("tree"))
print(frequencySort("cccaaa"))
print(frequencySort("Aabb"))

eert
aaaccc
bbAa


In [17]:
"""
456. 132模式
https://leetcode-cn.com/problems/132-pattern/
解题思路
1. 使用堆栈存储j的值, 使用变量s3存储k的值, 遍历数组, 得到i < k即可.
"""
def find132pattern(nums) -> bool:
    stack, s3 = [], float("-inf")
    for i in range(len(nums) - 1, -1, -1):
        if nums[i] < s3:
            return True
        while stack and stack[-1] < nums[i]:
            s3 = stack.pop()
        stack.append(nums[i])
    return False

# 简单测试
# False
print(find132pattern([1,2,3,4]))
# True
print(find132pattern([3,1,4,2]))
# True
print(find132pattern([-1,3,2,0]))

False
True
True


In [1]:
"""
503. 下一个更大元素II
https://leetcode-cn.com/problems/next-greater-element-ii/
解题思路
  1. 暴力解法(超出时间限制)
  2. 使用堆栈存储索引
"""
def nextGreaterElements(nums):
    arr = nums * 2
    r = []
    for i in range(len(nums)):
        for j in range(i + 1, len(arr)):
            if arr[j] > nums[i]:
                r.append(arr[j])
                break
        else:
            r.append(-1)
    
    return r

def nextGreaterElements1(nums):
    res, stack = [0 for i in range(len(nums))], []
    for i in range(len(nums) * 2 - 1, -1, -1):
        while stack and nums[stack[-1]] <= nums[i % len(nums)]:
            stack.pop()
        res[i % len(nums)] = nums[stack[-1]] if stack else -1
        stack.append(i % len(nums))
    
    return res

# 简单测试
# [2, -1, 2]
print(nextGreaterElements([1,2,1]))
# [3, 4, 4, 5, 6, 7, 6, 6, 7, 8, 9, 8, 7, 6, 7, 8, 9, -1, 3]
print(nextGreaterElements([1,3,2,4,5,6,5,4,6,7,8,7,6,5,6,7,8,9,1]))
# [2, 3, 4, 5, -1]
print(nextGreaterElements([1,2,3,4,5]))

# [2, -1, 2]
print(nextGreaterElements1([1,2,1]))
# [3, 4, 4, 5, 6, 7, 6, 6, 7, 8, 9, 8, 7, 6, 7, 8, 9, -1, 3]
print(nextGreaterElements1([1,3,2,4,5,6,5,4,6,7,8,7,6,5,6,7,8,9,1]))
# [2, 3, 4, 5, -1]
print(nextGreaterElements1([1,2,3,4,5]))

[7, -1, 5, 5, 7]
[2, -1, 2]
[3, 4, 4, 5, 6, 7, 6, 6, 7, 8, 9, 8, 7, 6, 7, 8, 9, -1, 3]
[2, 3, 4, 5, -1]
[2, -1, 2]
[3, 4, 4, 5, 6, 7, 6, 6, 7, 8, 9, 8, 7, 6, 7, 8, 9, -1, 3]
[2, 3, 4, 5, -1]


In [45]:
"""
621. 任务调度器
https://leetcode-cn.com/problems/task-scheduler/
"""
def leastInterval(tasks, n):
    arr = [0 for _ in range(26)]
    for t in tasks:
        arr[ord(t) - ord('A')] += 1
    arr.sort()
    max_val = arr[25] - 1
    idle_slots = max_val * n
    i = 24
    while i >= 0 and arr[i] > 0:
        idle_slots -= min(arr[i], max_val)
        i -= 1
    return idle_slots + len(tasks) if idle_slots > 0 else len(tasks)

print(leastInterval(['A', 'A', 'A', 'B', 'B', 'B'], 2))

8


In [44]:
"""
622. 设计循环队列
https://leetcode-cn.com/problems/design-circular-queue/
"""
class MyCircularQueue:

    def __init__(self, k: int):
        self.k = k
        self.stack = []

    def enQueue(self, value: int) -> bool:
        if self.isFull():
            return False
        self.stack.append(value)
        return True
        

    def deQueue(self) -> bool:
        if self.isEmpty():
            return False
        self.stack.pop(0)
        return True

    def Front(self) -> int:
        if self.isEmpty():
            return -1
        return self.stack[0]

    def Rear(self) -> int:
        if self.isEmpty():
            return -1
        return self.stack[-1]
        

    def isEmpty(self) -> bool:
        return bool(not self.stack)
        

    def isFull(self) -> bool:
        return len(self.stack) == self.k

In [59]:
"""
636. 函数的独占时间
https://leetcode-cn.com/problems/exclusive-time-of-functions/
答案来自官网
"""
def exclusiveTime(n, logs):
    stack = []
    res = [0 for i in range(n)]
    s = logs[0].split(":")
    stack.append(int(s[0]))
    i, prev = 1, int(s[2])
    while i < len(logs):
        s = logs[i].split(":")
        if s[1] == 'start':
            if stack:
                res[stack[-1]] += int(s[2]) - prev
            stack.append(int(s[0]))
            prev = int(s[2])
        else:
            res[stack[-1]] += int(s[2]) - prev + 1
            stack.pop()
            prev = int(s[2]) + 1
        i += 1
    return res

# 简单测试
# [3,4]
print(exclusiveTime(2, ["0:start:0","1:start:2","1:end:5","0:end:6"]))

[3, 4]


In [43]:
"""
641. 设计循环双端队列
https://leetcode-cn.com/problems/design-circular-deque/
"""
class MyCircularDeque:
    def __init__(self, k: int):
        self.stack = []
        self.k = k
    
    def insertFront(self, value: int) -> bool:
        if self.isFull():
            return False
        self.stack = [value] + self.stack
        return True
    
    def insertLast(self, value: int) -> bool:
        if self.isFull():
            return False
        self.stack.append(value)
        return True
    
    def deleteFront(self) -> bool:
        if self.isEmpty():
            return False
        self.stack.pop(0)
        return True
    
    def deleteLast(self) -> bool:
        if self.isEmpty():
            return False
        self.stack.pop()
        return True
    
    def getFront(self) -> int:
        if self.isEmpty():
            return -1
        return self.stack[0]
    
    def getRear(self) -> int:
        if self.isEmpty():
            return -1
        return self.stack[-1]
    
    def isEmpty(self) -> bool:
        return bool(not self.stack)
    
    def isFull(self) -> bool:
        return len(self.stack) == self.k

In [13]:
"""
659. 分割数组为连续子序列
https://leetcode-cn.com/problems/split-array-into-consecutive-subsequences/
"""
# 贪心算法
def isPossible(nums):
    count, tails = collections.Counter(nums), collections.Counter()
    for x in nums:
        if count[x] == 0:
            continue
        elif tails[x] > 0:
            tails[x] -= 1
            tails[x + 1] += 1
        elif count[x + 1] > 0 and count[x + 2] > 0:
            count[x + 1] -= 1
            count[x + 2] -= 1
            tails[x + 3] += 1
        else:
            return False
        count[x] -= 1
    return True

print(isPossible([1,2,3,3,4,5]))
print(isPossible([1,2,3,3,4,4,5,5]))
print(isPossible([1,2,3,4,4,5]))

True
True
False


In [14]:
"""
692. 前K个高频单词
https://leetcode-cn.com/problems/top-k-frequent-words/submissions/
"""
def topKFrequent(words, k):
        d, s = collections.defaultdict(int), []
        for w in words:
            d[w] += 1
        # 因为按照次数, 以及字母的自然顺序排序, 所以使用最小堆
        # 因为次数和字母自然顺序是冲突的, 一个降序, 一个升序, 所以才需要"-value"
        for key, value in d.items():
            s.append((-value, key))
        return [h[1] for h in heapq.nsmallest(k, s)]
    
print(topKFrequent(["i", "love", "leetcode", "i", "love", "coding"], 2))
print(topKFrequent(["the", "day", "is", "sunny", "the", "the", "the", "sunny", "is", "is"], 4))

['i', 'love']
['the', 'is', 'sunny', 'day']


In [41]:
"""
735. 行星碰撞
https://leetcode-cn.com/problems/asteroid-collision/
"""
def asteroidCollision(asteroids):
    stack = []
    for a in asteroids:
        # 堆栈为空
        if not stack:
            stack.append(a)
            continue
        if a > 0:
            # 因为正数向右边移动, 所以正数直接写入堆栈中
            stack.append(a)
        else:
            # 堆栈中存在负数, 不处理
            if stack[-1] < 0:
                stack.append(a)
            else:
                # 为False说明需要压入堆栈
                status = False
                while stack and stack[-1] > 0:
                    if a + stack[-1] > 0:
                        status = True
                        break
                    elif a + stack[-1] == 0:
                        stack.pop()
                        status = True
                        break
                    else:
                        stack.pop()
                if not status:
                    stack.append(a)
                    
    return stack

# 简单测试
print(asteroidCollision([5,10,-5]))
print(asteroidCollision([8,-8]))
print(asteroidCollision([10, 2, -5]))
print(asteroidCollision([-2, -1, 1, 2]))
print(asteroidCollision([-2, -2, 1, -2]))

[5, 10]
[]
[10]
[-2, -1, 1, 2]
[-2, -2, -2]


In [39]:
"""
739. 每日温度
https://leetcode-cn.com/problems/daily-temperatures/
解题思路
    1. 暴力破解(超时)
    2. 使用堆栈
"""
def dailyTemperatures(T):
    r = []
    for i in range(len(T) - 1):
        for j in range(i + 1, len(T)):
            if T[j] > T[i]:
                r.append(j - i)
                break
        else:
            r.append(0)
    r.append(0)
    return r

def dailyTemperatures1(T):
    # 堆栈存储大于当前值的(值, 索引)
    stack, r = [(T[-1], len(T) - 1)], [0]
    for i in range(len(T) - 2, -1, -1):
        while stack and T[i] >= stack[-1][0]:
            stack.pop()
        if not stack:
            r.append(0)
        else:
            r.append(stack[-1][1] - i)
        stack.append((T[i], i))
    r.reverse()
    return r

# 简单测试
print(dailyTemperatures([73, 74, 75, 71, 69, 72, 76, 73]))
print(dailyTemperatures1([73, 74, 75, 71, 69, 72, 76, 73]))

[1, 1, 4, 2, 1, 1, 0, 0]
[1, 1, 4, 2, 1, 1, 0, 0]


In [22]:
"""
743. 网络延迟时间
https://leetcode-cn.com/problems/network-delay-time/
"""
def networkDelayTime(times, N, K):
    graph = collections.defaultdict(list)
    for u, v, w in times:
        graph[u].append((w, v))
    
    dist = {node: float("inf") for node in range(1, N + 1)}
    
    def dfs(node, elapsed):
        if elapsed >= dist[node]: return
        dist[node] = elapsed
        for time, nei in sorted(graph[node]):
            dfs(nei, elapsed + time)
    
    dfs(K, 0)
    ans = max(dist.values())
    return ans if ans < float("inf") else -1

def networkDelayTime1(times, N, K):
    graph = collections.defaultdict(list)
    for u, v, w in times:
        graph[u].append((v, w))
    
    pq = [(0, K)]
    dist = {}
    while pq:
        d, node = heapq.heappop(pq)
        if node in dist:
            continue
        dist[node] = d
        for nei, d2 in graph[node]:
            if nei not in dist:
                heapq.heappush(pq, (d + d2, nei))
    
    return max(dist.values()) if len(dist) == N else -1

print(networkDelayTime([[2,1,1],[2,3,1],[3,4,1]], 4, 2))
print(networkDelayTime1([[2,1,1],[2,3,1],[3,4,1]], 4, 2))

2
2


In [24]:
"""
767. 重构字符串
https://leetcode-cn.com/problems/reorganize-string/submissions/
"""
def reorganizeString(S):
    N, A = len(S), []
    for c, x in sorted((S.count(x), x) for x in set(S)):
        if c > (N + 1) / 2:
            return ""
        A.extend(c * x)
    ans = [None] * N
    ans[::2], ans[1::2] = A[N//2:], A[:N//2]
    return "".join(ans)

print(reorganizeString("todrnphcamnomskfrhe"))

hahcmdmenfnkoposrtr


In [ ]:
"""
787. K 站中转内最便宜的航班
https://leetcode-cn.com/problems/cheapest-flights-within-k-stops/
"""
def findCheapestPrice(n, flights, src, dst, k):
    f = collecctions.defaultdict(dict)
    for a, b, p in flights:
        f[a][b] = p
    heap = [(0, src, k + 1)]
    while heap:
        p, i, k = heapq.heappop(heap)
        if i == dst:
            return p
        if k > 0:
            for j in f[i]:
                heapq.heappush(heap, (p + f[i][j], j, k - 1))
    return -1


In [33]:
"""
856. 括号的分数
https://leetcode-cn.com/problems/score-of-parentheses/
"""
def scoreOfParentheses(S: str) -> int:
    def help(s):
        if not s:
            return 0
        # count用于判断左右括号个数是否相等. 等于0则代表相等.
        # index用于定位count==0时候的索引.
        count, index = 1, 1
        while index < len(s):
            if s[index] == '(':
                count += 1
            else:
                count -= 1
            if count == 0:
                break
            index += 1
        # 字符以()开头
        if index == 1:
            return 1 + help(s[2:])
        else:
            return 2 * help(s[1:index]) + help(s[index + 1:])
    return help(S)
        

# 简单测试
print(scoreOfParentheses("()"))
print(scoreOfParentheses("(())"))
print(scoreOfParentheses("()()"))
print(scoreOfParentheses("(()(()))"))
            

1
2
2
6


In [32]:
"""
880. 索引处的解码字符串
https://leetcode-cn.com/problems/decoded-string-at-index/
解答来自LeetCode的官网
"""
def decodeAtIndex(S: str, K: int) -> str:
    size = 0
    # 获取字符串的总数
    for s in S:
        if s.isdigit():
            size *= int(s)
        else:
            size += 1
    
    for s in reversed(S):
        K %= size
        # 匹配字符
        if K == 0 and s.isalpha():
            return s
        if s.isdigit():
            size //= int(s)
        else:
            size -= 1
            
            
# 简单测试
# o
print(decodeAtIndex("leet2code3", 10))
# h
print(decodeAtIndex("ha22", 5))
# a
print(decodeAtIndex("a2345678999999999999999", 1))

o
h
a


In [31]:
"""
901. 股票价格跨度
https://leetcode-cn.com/problems/online-stock-span/
解题思路
    1. 暴力破解(超时)
    2. 将次数存储起来
"""
class StockSpanner:
    def __init__(self):
        self.stack = []
    
    def next(self, price: int) -> int:
        self.stack.append(price)
        length, i = len(self.stack), len(self.stack) - 2
        while i >= 0:
            if self.stack[i] > self.stack[length - 1]:
                break
            i -= 1
        
        if i >= 0:
            return 1 + length - 2 - i
        return length
    
class StockSpanner1:
    def __init__(self):
        self.stack = []
    
    def next(self, price: int) -> int:
        weight = 1
        while self.stack and self.stack[-1][0] <= price:
            weight += self.stack.pop()[1]
        # 堆栈存储价格和次数
        self.stack.append((price, weight))
        return weight
        
# 简单测试
s = StockSpanner()
print(s.next(100))
print(s.next(80))
print(s.next(60))
print(s.next(70))
print(s.next(60))
print(s.next(75))
print(s.next(85))

s1 = StockSpanner1()
print(s1.next(100))
print(s1.next(80))
print(s1.next(60))
print(s1.next(70))
print(s1.next(60))
print(s1.next(75))
print(s1.next(85))

1
1
1
2
1
4
6
1
1
1
2
1
4
6


In [30]:
"""
907. 子数组的最小值之和
https://leetcode-cn.com/problems/sum-of-subarray-minimums/
解题思路
    1. 来自官方解答, 但是没有看懂, 先记下
    https://leetcode.com/problems/sum-of-subarray-minimums/solution/
"""
def sumSubarrayMins(A):
    MOD = 10 ** 9 + 7
    
    stack, ans, dot = [], 0, 0
    for j, y in enumerate(A):
        count = 1
        while stack and stack[-1][0] >= y:
            x, c = stack.pop()
            count += c
            dot -= x * c
        
        stack.append((y, count))
        dot += y * count
        ans += dot
    return ans % MOD

# 简单测试
# 17
print(sumSubarrayMins([3,1,2,4]))

17


In [29]:
"""
921. 使括号有效的最小添加
https://leetcode-cn.com/problems/minimum-add-to-make-parentheses-valid/
解题思路:
1. 定义stack, 用于存储消除()后的无效左括号.
2. r用于存储无效的右括号.
"""

def minAddToMakeValid(S: str) -> int:
    stack, r = [], 0
    for s in S:
        if s == '(':
            stack.append(s)
        else:
            if stack and stack[-1] == '(':
                stack.pop()
            else:
                r += 1
    return r + len(stack)

print(minAddToMakeValid("())"))
print(minAddToMakeValid("((("))
print(minAddToMakeValid("()"))
print(minAddToMakeValid("()))(("))

1
3
0
4


In [27]:
"""
946. 验证栈序列
https://leetcode-cn.com/problems/validate-stack-sequences/
解题思路:
1. 使用堆栈存储pushed中当前不可popped匹配的元素.
"""
def validateStackSequences(pushed, popped) -> bool:
    stack = []
    while pushed and popped:
        v = pushed.pop(0)
        # 如果pushed和popped相等, 则相消
        if v == popped[0]:
            popped.pop(0)
            while stack and stack[-1] == popped[0]:
                stack.pop()
                popped.pop(0)
        else:
            # 不匹配的, 先存入stack中
            stack.append(v)
    
    return not stack

# 简单测试
print(validateStackSequences([1,2,3,4,5], [4,5,3,2,1]))
print(validateStackSequences([1,2,3,4,5], [4,3,5,1,2]))

True
False


In [25]:
"""
973. 最接近原点的K个点
https://leetcode-cn.com/problems/k-closest-points-to-origin/
"""
def kClosest(points, K):
    if len(points) <= K:
        return points

    r = []
    for p in points:
        heapq.heappush(r, (p[0] ** 2 + p[1] ** 2, p))
    return [heapq.heappop(r)[1] for _ in range(K)]

print(kClosest([[1,3],[-2,2]], 1))
print(kClosest([[3,3],[5,-1],[-2,4]], 2))

[[-2, 2]]
[[3, 3], [-2, 4]]


In [26]:
"""
1003. 检查替换后的词是否有效
https://leetcode-cn.com/problems/check-if-word-is-valid-after-substitutions/
"""
def isValid(S: str) -> bool:
    while S:
        if 'abc' not in S:
            return False
        else:
            index = S.index('abc')
            S = S[0:index] + S[index + 3:]
    return True

# 简单测试
print(isValid("aabcbc"))
print(isValid("abcabcababcc"))
print(isValid("abccba"))
print(isValid("cababc"))

True
True
False
False


In [25]:
"""
1019. 链表中的下一个更大节点
https://leetcode-cn.com/problems/next-greater-node-in-linked-list/
解题思路:
1. 定义堆栈stack, 元素存储链表的值和索引.
2. 不断遍历链表, 如果当前链表的值大于堆栈的栈顶, 则当前链表值就是栈顶元素对应的链表值得下一个更大节点.
"""
def nextLargerNodes(head: ListNode):
    stack, ans = [(float("inf"), -1)], []
    index = -1
    while head:
        ans.append(0)
        index += 1
        while head.val > stack[-1][0]:
            ans[stack[-1][1]] = head.val
            stack.pop()
        stack.append((head.val, index))
        head = head.next
    return ans
    
t1, t2, t3 = array2link([2,1,5]), array2link([2,7,4,3,5]), array2link([1,7,5,1,9,2,5,1])
print(nextLargerNodes(t1))
print(nextLargerNodes(t2))
print(nextLargerNodes(t3))

[5, 5, 0]
[7, 0, 5, 5, 0]
[7, 9, 9, 9, 0, 5, 0, 0]


In [31]:
"""
1054. 距离相等的条形码
https://leetcode-cn.com/problems/distant-barcodes/
"""
def rearrangeBarcodes(barcodes):
    N = len(barcodes)
    count = collections.Counter(barcodes)
    barcodes.sort(key=lambda a: (count[a], a))
    ans = [None] * N
    ans[::2], ans[1::2] = barcodes[N // 2:], barcodes[:N // 2]
    return ans

print(rearrangeBarcodes([1,1,1,2,2,2]))
print(rearrangeBarcodes([1,1,1,1,2,2,3,3]))

[2, 1, 2, 1, 2, 1]
[1, 2, 1, 2, 1, 3, 1, 3]


In [22]:
"""
1124. 表现良好的最长时间段
https://leetcode-cn.com/problems/longest-well-performing-interval/
解题思路
  1. 暴力解法
  2. 使用字典(来自: https://leetcode.com/problems/longest-well-performing-interval/discuss/334565/JavaC%2B%2BPython-O(N)-Solution-Life-needs-996-and-669)
"""
def longestWPI(hours) -> int:
    r = 0
    # 将大于8的设为1, 小于等于8的设为-1
    for i in range(len(hours)):
        hours[i] = 1 if hours[i] > 8 else -1
    
    # 循环列表, 判断和大于0, 则代表大于8的多, 则为"表现良好时间段"
    for i in range(len(hours)):
        sum = 0
        for j in range(i, len(hours)):
            sum += hours[j]
            if sum > 0:
                r = max(r, j - i + 1)
    
    return r

def longestWPI1(hours) -> int:
    res, score, seen = 0, 0, {}
    for i, h in enumerate(hours):
        score = score + 1 if h > 8 else score - 1
        if score > 0:
            res = i + 1
        seen.setdefault(score, i)
        # 精髓所在: 如果score - 1在之前已经出现过, 则长度可以为: i - seen[score - 1].
        # 然后取res, i - seen[score - 1]中的最大值
        if score - 1 in seen:
            res = max(res, i - seen[score - 1])
    
    return res

# 简单测试
print(longestWPI([9,9,6,0,6,6,9]))
print(longestWPI([6,6,9]))
print(longestWPI([6,9,6]))

print(longestWPI1([9,9,6,0,6,6,9]))
print(longestWPI1([6,6,9]))
print(longestWPI1([6,9,6]))  

3
1
1
3
1
1
